In [17]:
from __future__ import annotations

from pathlib import Path

import pandas as pd

from run import Dataset

DATASET = Dataset.TOPN_US.value(topn=500)

In [18]:
SELECTED_FACTORS = ["low_risk", "momentum", "size", "quality", "value"]

jkp_factors = pd.read_csv(Path("../../data/jkp_raw") / "jkp_factors.csv")
jkp_factors = jkp_factors[jkp_factors["name"].isin(SELECTED_FACTORS)]
jkp_factors["date"] = pd.to_datetime(jkp_factors["date"])

In [19]:
factors = jkp_factors.pivot_table(index="date", columns="name", values="ret")
factors

name,low_risk,momentum,quality,size,value
date,,,,,
1926-01-02,NaN,NaN,NaN,0.002354,NaN
1926-01-04,NaN,NaN,NaN,0.005459,NaN
1926-01-05,NaN,NaN,NaN,0.003777,NaN
1926-01-06,NaN,NaN,NaN,-0.000590,NaN
1926-01-07,NaN,NaN,NaN,0.002057,NaN
...,...,...,...,...,...
2024-12-24,-0.003400,0.001616,-0.000962,0.001315,-0.002292
2024-12-26,-0.003880,-0.000633,-0.002414,0.008669,-0.001183
2024-12-27,0.007210,-0.004166,0.000425,-0.003512,0.005444


In [20]:
data_df = pd.read_csv(Path("../../data/output") / DATASET.DF_FILENAME)
data_df["date"] = pd.to_datetime(data_df["date"])
data_df = data_df.set_index("date")

In [21]:
spx = pd.read_excel(Path("../../data/gw_replication") / "spx.xlsx", skiprows=6)
spx = spx.rename(columns={"Date": "date", "PX_LAST": "spx"})
spx["date"] = pd.to_datetime(spx["date"])
spx = spx.set_index("date")
spx = spx.sort_index()
spx = spx[["spx"]].pct_change()
spx

,spx
date,
1950-01-30,NaN
1950-01-31,0.001763
1950-02-01,0.000000
1950-02-02,0.010557
1950-02-03,0.003482
...,...
2025-01-22,0.006138
2025-01-23,0.005314
2025-01-24,-0.002855


In [22]:
rf = pd.read_excel(Path("../../data/ff") / "FFDaily.xlsx")
rf = rf.rename(columns={"Date": "date", "RF": "rf"})
rf["date"] = pd.to_datetime(rf["date"], format="%Y%m%d")
rf = rf.set_index("date")
rf = rf["rf"] / 100
rf

date
1926-07-01    0.00009
1926-07-02    0.00009
1926-07-06    0.00009
1926-07-07    0.00009
1926-07-08    0.00009
               ...   
2024-08-26    0.00022
2024-08-27    0.00022
2024-08-28    0.00022
2024-08-29    0.00022
2024-08-30    0.00022
Name: rf, Length: 25817, dtype: float64

In [23]:
from qamsi.market_data.risk_free_conventions import accrue_risk_free_rate

rf = accrue_risk_free_rate(rf_rate=rf)

In [24]:
spx = spx.merge(rf.rename("acc_rate"), left_index=True, right_index=True, how="left")
spx

,spx,acc_rate
date,,
1950-01-30,NaN,0.00008
1950-01-31,0.001763,0.00004
1950-02-01,0.000000,0.00004
1950-02-02,0.010557,0.00004
1950-02-03,0.003482,0.00004
...,...,...
2025-01-22,0.006138,NaN
2025-01-23,0.005314,NaN
2025-01-24,-0.002855,NaN


In [25]:
spx = spx[["spx"]].sub(spx["acc_rate"], axis=0)
spx

,spx
date,
1950-01-30,NaN
1950-01-31,0.001723
1950-02-01,-0.000040
1950-02-02,0.010517
1950-02-03,0.003442
...,...
2025-01-22,NaN
2025-01-23,NaN
2025-01-24,NaN


In [26]:
factors = factors.merge(spx, left_index=True, right_index=True, how="left")

In [27]:
data_df = data_df.merge(factors, left_index=True, right_index=True, how="left")
data_df.shape

(11324, 2471)

In [28]:
data_df = data_df.merge(
    rf.rename("acc_rate"), left_index=True, right_index=True, how="left"
)
data_df.shape

(11324, 2472)

In [29]:
data_df.columns

Index(['10006', '10078', '10095', '10104', '10107', '10108', '10119', '10137',
       '10138', '10145',
       ...
       '93422', '93429', '93436', 'low_risk', 'momentum', 'quality', 'size',
       'value', 'spx', 'acc_rate'],
      dtype='object', length=2472)

In [30]:
# data_df[factors.columns].to_csv(Path("../../data/output") / "factors.csv")

In [31]:
# data_df[data_df.columns.difference(factors.columns.tolist() + ["spx"])].to_csv(Path("../../data/output") / "spx_data.csv")

In [32]:
data_df.to_csv(Path("../../data/output") / DATASET.DF_FILENAME)